In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def fetch_articles(page_number):
    url = f"https://www.yomiuri.co.jp/y_ajax/latest_list_news_more/////50//50/{page_number}/?action=latest_list_news_more&others=%2F%2F%2F%2F50%2F%2F50%2F{page_number}%2F"
    payload = {
        "action": "latest_list_news_more",
        "others": f"////50//50/{page_number}/"
    }

    response = requests.get(url, params=payload)
    
    if response.status_code == 200:
        data = response.json()
        content = data.get('contents', '')
        soup = BeautifulSoup(content, 'html.parser')
        articles = soup.find_all('article', class_='news-top-latest__list-item')
        return articles
    else:
        print(f"Failed to retrieve data for page {page_number}.")
        return []

def extract_article_info(article):
    title_tag = article.find('h3')
    link_tag = title_tag.find('a')
    title = title_tag.text.strip()
    link = "https://www.yomiuri.co.jp" + link_tag['href']
    
    date_tag = article.find('time')
    date = date_tag['datetime'] if date_tag else 'Unknown'
    
    category_tag = article.find('div', class_='c-matome-title')
    category = category_tag.text.strip() if category_tag else 'No Category'
    
    preview_tag = article.find('p', class_='news-top-latest__list-item__text')
    preview = preview_tag.text.strip() if preview_tag else 'No Preview'
    
    return {'Title': title, 'Link': link, 'Date': date, 'Category': category, 'Preview': preview}

all_articles = []
for page_number in range(1, 6):  # Adjust the range as needed
    articles = fetch_articles(page_number)
    for article in articles:
        article_info = extract_article_info(article)
        all_articles.append(article_info)

df = pd.DataFrame(all_articles)
df